In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import shutil
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dropout,Dense,Conv2D,MaxPooling2D,GlobalAveragePooling2D

In [2]:
test_data= pd.read_csv('../input/digit-recognizer/test.csv')
train_data= pd.read_csv('../input/digit-recognizer/train.csv')
y = train_data.iloc[:,:1]
x = train_data.iloc[:,1:]
x/=255.0
x=x.values.reshape(-1,28,28,1)
print(x.shape)

(42000, 28, 28, 1)


In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (28,28,1)),
#     tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',padding = 'same'),
    tf.keras.layers.MaxPooling2D(2,2),
#         tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64,(1,1),activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'same'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024,activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512,activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128,activation = 'relu'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(10,activation = 'softmax')
])
model.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 13, 13, 64)        2112      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 3, 3, 64)         

In [7]:
history = model.fit(x,y,epochs = 100,verbose = 1)

Train on 42000 samples
Epoch 1/100
42000/42000 [==============================] - 6s 133us/sample - loss: 0.1708 - accuracy: 0.9498
Epoch 2/100
42000/42000 [==============================] - 6s 136us/sample - loss: 0.0777 - accuracy: 0.9784
Epoch 3/100
42000/42000 [==============================] - 6s 139us/sample - loss: 0.0627 - accuracy: 0.9835
Epoch 4/100
42000/42000 [==============================] - 6s 135us/sample - loss: 0.0508 - accuracy: 0.9868
Epoch 5/100
42000/42000 [==============================] - 6s 138us/sample - loss: 0.0440 - accuracy: 0.9882
Epoch 6/100
42000/42000 [==============================] - 5s 129us/sample - loss: 0.0414 - accuracy: 0.9894
Epoch 7/100
42000/42000 [==============================] - 5s 130us/sample - loss: 0.0378 - accuracy: 0.9901
Epoch 8/100
42000/42000 [==============================] - 5s 127us/sample - loss: 0.0342 - accuracy: 0.9913
Epoch 9/100
42000/42000 [==============================] - 5s 130us/sample - loss: 0.0292 - accuracy: 0.9

42000/42000 [==============================] - 5s 127us/sample - loss: 0.0208 - accuracy: 0.9972
Epoch 76/100
42000/42000 [==============================] - 6s 137us/sample - loss: 0.0213 - accuracy: 0.9964
Epoch 77/100
42000/42000 [==============================] - 6s 138us/sample - loss: 0.0161 - accuracy: 0.9977
Epoch 78/100
42000/42000 [==============================] - 6s 139us/sample - loss: 0.0199 - accuracy: 0.9974
Epoch 79/100
42000/42000 [==============================] - 6s 133us/sample - loss: 0.0258 - accuracy: 0.9970
Epoch 80/100
42000/42000 [==============================] - 6s 132us/sample - loss: 0.0176 - accuracy: 0.9976
Epoch 81/100
42000/42000 [==============================] - 5s 128us/sample - loss: 0.0165 - accuracy: 0.9970
Epoch 82/100
42000/42000 [==============================] - 6s 135us/sample - loss: 0.0237 - accuracy: 0.9967
Epoch 83/100
42000/42000 [==============================] - 5s 127us/sample - loss: 0.0260 - accuracy: 0.9965
Epoch 84/100
42000/4200

In [8]:
test_data = test_data.values.reshape(-1,28,28,1)

In [10]:
test_data = test_data / 255.0

In [11]:
submit = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
submit.Label =model.predict_classes(test_data)
submit.head()
submit.to_csv('submit.csv',index=False)